# 文本分类实例

## Step1 导入相关包

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

## Step2 加载数据集

In [3]:
data = load_dataset('csv', data_files='./datasets/ChnSentiCorp_htl_all.csv', split='train')
data = data.filter(lambda x: x['review'] is not None)
data

Generating train split: 7766 examples [00:00, 119280.81 examples/s]
Filter: 100%|██████████| 7766/7766 [00:00<00:00, 174742.98 examples/s]


Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

## Step3 划分数据集

In [4]:
data_split = data.train_test_split(test_size=0.2)
data_split

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6212
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 1553
    })
})

## Step4 数据集预处理

In [7]:
import torch

tokenizer = AutoTokenizer.from_pretrained("D:/pretrained_model/models--hfl--rbt3")

def process_function(examples):
    tokenized_examples = tokenizer(examples['review'], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples['label']
    return tokenized_examples

tokenizer_datasets = data_split.map(process_function, batched=True)

loading configuration file D:/pretrained_model/models--hfl--rbt3\config.json
Model config BertConfig {
  "_name_or_path": "D:/pretrained_model/models--hfl--rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

loading file vocab.txt
loading file tokenizer.json
loading file 

## Step5 创建模型

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("D:/pretrained_model/models--hfl--rbt3")

loading configuration file D:/pretrained_model/models--hfl--rbt3\config.json
Model config BertConfig {
  "_name_or_path": "D:/pretrained_model/models--hfl--rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

loading weights file D:/pretrained_model/models--hfl--rbt3\pytor

In [9]:
model.config

BertConfig {
  "_name_or_path": "D:/pretrained_model/models--hfl--rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

## Step6 创建评估函数

In [10]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [11]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## Step7 创建TrainingArguments

In [15]:
train_args = TrainingArguments(
    output_dir='./checkpoints',
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    num_train_epochs=5
)
train_args

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_toke

## Step8 创建Trainer

In [16]:
from transformers import DataCollatorWithPadding

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenizer_datasets['train'],
    eval_dataset=tokenizer_datasets['test'],
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=eval_metric
)

## Step9 模型训练

In [17]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review. If review are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
d:\anaconda3\envs\py39\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6,212
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 490
  Number of trainable parameters = 38,478,338
 20%|█▉        | 97/490 [00:25<01:44,  3.76it/s]The following columns in the evaluation set don't have a corresponding argument in `B

{'eval_loss': 0.2863149642944336, 'eval_accuracy': 0.8898905344494527, 'eval_f1': 0.919605077574048, 'eval_runtime': 2.3145, 'eval_samples_per_second': 670.979, 'eval_steps_per_second': 5.617, 'epoch': 1.0}


Model weights saved in ./checkpoints\checkpoint-98\pytorch_model.bin
 20%|██        | 100/490 [00:28<04:44,  1.37it/s]

{'loss': 0.2219, 'learning_rate': 1.5918367346938776e-05, 'epoch': 1.02}


 40%|███▉      | 195/490 [00:53<01:13,  4.04it/s]The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review. If review are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1553
  Batch size = 128
                                                 
 40%|████      | 196/490 [00:55<01:12,  4.04it/s]Saving model checkpoint to ./checkpoints\checkpoint-196
Configuration saved in ./checkpoints\checkpoint-196\config.json


...

100%|█████████▉| 489/490 [02:17<00:00,  3.98it/s]The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review. If review are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1553
  Batch size = 128
                                                 
100%|██████████| 490/490 [02:19<00:00,  3.98it/s]Saving model checkpoint to ./checkpoints\checkpoint-490
Configuration saved in ./checkpoints\checkpoint-490\config.json


{'eval_loss': 0.29238489270210266, 'eval_accuracy': 0.8956857694784288, 'eval_f1': 0.9230038022813688, 'eval_runtime': 2.3751, 'eval_samples_per_second': 653.857, 'eval_steps_per_second': 5.473, 'epoch': 5.0}


Model weights saved in ./checkpoints\checkpoint-490\pytorch_model.bin
Deleting older checkpoint [checkpoints\checkpoint-196] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./checkpoints\checkpoint-392 (score: 0.923943661971831).
d:\anaconda3\envs\py39\lib\site-packages\transformers\trainer.py:2245: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted

{'train_runtime': 140.3712, 'train_samples_per_second': 221.271, 'train_steps_per_second': 3.491, 'train_loss': 0.16932123923788264, 'epoch': 5.0}


TrainOutput(global_step=490, training_loss=0.16932123923788264, metrics={'train_runtime': 140.3712, 'train_samples_per_second': 221.271, 'train_steps_per_second': 3.491, 'train_loss': 0.16932123923788264, 'epoch': 5.0})

## Step10 模型评估

In [18]:
trainer.evaluate(tokenizer_datasets['test'])

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review. If review are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1553
  Batch size = 128
100%|██████████| 13/13 [00:02<00:00,  6.28it/s]


{'eval_loss': 0.2884453237056732,
 'eval_accuracy': 0.8956857694784288,
 'eval_f1': 0.923943661971831,
 'eval_runtime': 2.298,
 'eval_samples_per_second': 675.796,
 'eval_steps_per_second': 5.657,
 'epoch': 5.0}

## Step11 模型预测

In [19]:
trainer.predict(tokenizer_datasets['test'])

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review. If review are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1553
  Batch size = 128
100%|██████████| 13/13 [00:02<00:00,  6.12it/s]


PredictionOutput(predictions=array([[-3.03304   ,  2.5443125 ],
       [-0.77071697,  0.64754   ],
       [-2.9726257 ,  2.0428975 ],
       ...,
       [-2.312079  ,  1.7297918 ],
       [-3.6496766 ,  3.3320827 ],
       [-1.6093348 ,  1.066688  ]], dtype=float32), label_ids=array([1, 0, 1, ..., 1, 1, 1], dtype=int64), metrics={'test_loss': 0.2884453237056732, 'test_accuracy': 0.8956857694784288, 'test_f1': 0.923943661971831, 'test_runtime': 2.3901, 'test_samples_per_second': 649.773, 'test_steps_per_second': 5.439})

In [20]:
from transformers import pipeline

id2_label = {0: "差评！", 1: "好评！"}

model.config.id2label = id2_label

pipeline_model = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [24]:
sen = '我觉得就那样'
pipeline_model(sen)

[{'label': '好评！', 'score': 0.7128965258598328}]